# gcm_filters Test

This notebook is for learning how to use the `gcm_filters` package to smooth gridded data. Dhruv said this package is adventageous because it handles boundaries well and also returns a dataset on the same grid it was provided on.

The goal here is to do 2 things to the diffusivity datasets from Argo: 
1. fill in the gaps of the diffusivity dataset and expand the spatial extent farther north and south
2. smooth the diffusivity dataset (there's more information here than we need)

To fill in gaps, you should use the median value on that density surfce. Also make sure that every point everywhere has a value in the diffusivity dataset, that way the Argo gradient data is what determines where there will be values. 

After this, use gcm_filters to smooth (regular filters with nans? is this the name of a function or an example notebook or what?).

In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.path import Path
import matplotlib.colors as colors
import seaborn as sns
import seaborn
import pandas as pd
import numpy as np
from importlib import reload
import cartopy.crs as ccrs
import cmocean.cm as cmo
import gsw
import scipy.ndimage as filter
import scipy.interpolate as interpolate
from flox.xarray import xarray_reduce
from xgcm.autogenerate import generate_grid_ds
from xgcm import Grid

In [41]:
ds_diff = xr.open_dataset('/swot/SUM05/amf2288/global_diff.nc')

In [42]:
ds_binned = xr.open_dataset('/swot/SUM05/amf2288/sync-boxes/globe_binned_3_z.nc')

In [51]:
ds_diff = ds_diff.interp(z_c=ds_binned.z_c)

In [54]:
ds_diff = generate_grid_ds(ds_diff, {'X':'lon_c', 'Y':'lat_c'})
grid = Grid(ds_diff, coords={
        'X': {'center': 'lon_c', 'left': 'lon_c_left'},
        'Y': {'center': 'lat_c', 'left': 'lat_c_left'},
        'Z': {'center': 'z_c'}
        }, periodic=['X'])

In [55]:
ds_diff = ds_diff.rename({'lon_c':'lon', 'lon_c_left':'lon_left',
                                'lat_c':'lat', 'lat_c_left':'lat_left',
                                'z_c':'z'})
ds_diff = ds_diff.drop_vars(['z_l', 'z_r', 'lon_l', 'lon_r', 'lat_l', 'lat_r'])

Now I need to:
- expand lats to whole globe
- fill in nans with median value on depth level
- smooth